Script used to use rapidAPI to scrape zillow addresses at scale

In [ ]:
#sample use on first address
import requests

url = "https://zillow56.p.rapidapi.com/search_address"

querystring = {"address":"5460 W PRENTICE CIR Denver"}

headers = {
	"X-RapidAPI-Key": "a11fbe76c2msh84598c4ddde61a5p107480jsn9e228e26d6e4",
	"X-RapidAPI-Host": "zillow56.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

#print(response.json())
print(response.json()['resoFacts']['atAGlanceFacts'][1]['factValue'])

1998


In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv('/content/CrossRegionalData (2).csv')

df.head()

,Address,Lead,No Lead,City,State
0,5460 W PRENTICE CIR,0,1,Denver,Colorado
1,1119 S ALTON CT,0,1,Denver,Colorado
2,1995 E OTERO LN,0,1,Denver,Colorado
3,44 COVINGTON CT,0,1,Denver,Colorado
4,1115 GAYLORD ST,1,0,Denver,Colorado


In [ ]:
#checking df
df.loc[df['City'] == 'Cincinnati', 'City'] = 'Grand Rapids'
df.loc[df['State'] == 'Ohio', 'State'] = 'Michigan'
num_gr = df[df['City']=='Grand Rapids'].shape[0]
shuffled_df = df.sample(frac=1, random_state=42).reset_index(drop=True)
shuffled_df.shape
shuffled_df.to_csv('shuffled.csv')

In [ ]:
import pandas as pd

shuffled_df = pd.read_csv('/content/drive/MyDrive/shuffled.csv')
address_df = shuffled_df[['Address', 'City']]
address_df.head()

,Address,City
0,5440 BRYANT ST,Denver
1,5414 W 17TH AVE,Denver
2,331 34TH PL NE,DC
3,4221 SHERROD ST SHERROD ST,Pittsburgh
4,2206 DAWSON AVE NE,Grand Rapids


In [ ]:
import json
import csv
import requests
import random

def get_address(address_str, city):
    url = "https://zillow56.p.rapidapi.com/search_address"
    address = address_str + " " + city
    querystring = {"address": address}

    headers = {
        "X-RapidAPI-Key": "a11fbe76c2msh84598c4ddde61a5p107480jsn9e228e26d6e4",
        "X-RapidAPI-Host": "zillow56.p.rapidapi.com"
    }

    try:
        response = requests.get(url, headers=headers, params=querystring).json()

        # Create a dictionary to store the extracted data
        extracted_data = {
        'abbreviatedAddress': response.get('abbreviatedAddress', ''),
        'bathrooms': response.get('bathrooms', ''),
        'bedrooms': response.get('bedrooms', ''),
        'city': response.get('city', ''),
        'cityId': response.get('cityId', ''),
        'county': response.get('county', ''),
        'countyFIPS': response.get('countyFIPS', ''),
        'countyId': response.get('countyId', ''),
        'dateSold': response.get('dateSold', ''),
        'daysOnZillow': response.get('daysOnZillow', ''),
        'description': response.get('description', ''),
        'homeInsights': response.get('homeInsights', ''),
        'homeStatus': response.get('homeStatus', ''),
        'homeType': response.get('homeType', ''),
        'homeValues': response.get('homeValues', ''),
        'lastSoldPrice': response.get('lastSoldPrice', ''),
        'latitude': response.get('latitude', ''),
        'listPriceLow': response.get('listPriceLow', ''),
        'livingArea': response.get('livingArea', ''),
        'livingAreaUnits': response.get('livingAreaUnits', ''),
        'livingAreaUnitsShort': response.get('livingAreaUnitsShort', ''),
        'livingAreaValue': response.get('livingAreaValue', ''),
        'longitude': response.get('longitude', ''),
        'lotAreaUnits': response.get('lotAreaUnits', ''),
        'lotAreaValue': response.get('lotAreaValue', ''),
        'lotSize': response.get('lotSize', ''),
        'nearbyZipcodes': response.get('nearbyZipcodes', ''),
        'neighborhoodRegion': response.get('neighborhoodRegion', ''),
        'parcelId': response.get('parcelId', ''),
        'postingUrl': response.get('postingUrl', ''),
        'price': response.get('price', ''),
        'priceChange': response.get('priceChange', ''),
        'priceChangeDate': response.get('priceChangeDate', ''),
        'priceChangeDateString': response.get('priceChangeDateString', ''),
        'priceHistory': response.get('priceHistory', ''),
        'propertyTaxRate': response.get('propertyTaxRate', ''),
        'propertyTypeDimension': response.get('propertyTypeDimension', ''),
        'providerListingID': response.get('providerListingID', ''),
        'regionString': response.get('regionString', ''),
        'rentZestimate': response.get('rentZestimate', ''),
        'restimateHighPercent': response.get('restimateHighPercent', ''),
        'restimateLowPercent': response.get('restimateLowPercent', ''),
        'solarPotential': response.get('solarPotential', ''),
        'state': response.get('state', ''),
        'stateId': response.get('stateId', ''),
        'streetAddress': response.get('streetAddress', ''),
        'taxAssessedValue': response.get('taxAssessedValue', ''),
        'taxAssessedYear': response.get('taxAssessedYear', ''),
        'taxHistory': response.get('taxHistory', ''),
        'timeOnZillow': response.get('timeOnZillow', ''),
        'yearBuilt': response.get('yearBuilt', ''),
        'zestimate': response.get('zestimate', ''),
        'zestimateDeepDiveData': response.get('zestimateDeepDiveData', ''),
        'zestimateHighPercent': response.get('zestimateHighPercent', ''),
        'zestimateLowPercent': response.get('zestimateLowPercent', ''),
        'zipPlusFour': response.get('zipPlusFour', ''),
        'zipcode': response.get('zipcode', ''),
        'zpid': response.get('zpid', ''),
        'yearBuilt': response.get('resoFacts', '')
        }

        # CSV file header
        csv_header = extracted_data.keys()

        # Open the CSV file in 'append' mode
        with open('/content/drive/MyDrive/test.csv', mode='a', newline='') as csv_file:
            writer = csv.DictWriter(csv_file, fieldnames=csv_header)

            # Check if the file is empty and write the header if needed
            if csv_file.tell() == 0:
                writer.writeheader()

            # Write the extracted data to the CSV file
            writer.writerow(extracted_data)

    except Exception as e:
        print(f"Error processing address: {address_str}, {city}. Error: {e}")
        pass  # Continue to the next row even if an error occurs

 # Iterate over the selected rows and call get_address for each row
for i in range(122543,150000):
    get_address(address_df['Address'][i], address_df['City'][i])

Error processing address: 1932 MORRIS ST MORRIS ST, Pittsburgh. Error: Expecting value: line 1 column 1 (char 0)


In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
test_df = pd.read_csv('/content/drive/MyDrive/test.csv')
test_df.shape

<ipython-input-3-2ff57e402ef7>:3: DtypeWarning: Columns (56) have mixed types. Specify dtype option on import or set low_memory=False.
  test_df = pd.read_csv('/content/drive/MyDrive/test.csv')


(135613, 58)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
test_df['abbreviatedAddress'].isna().sum()


15265

In [ ]:
test_df.head(10)

,abbreviatedAddress,bathrooms,bedrooms,city,cityId,county,countyFIPS,countyId,dateSold,daysOnZillow,description,homeInsights,homeStatus,homeType,homeValues,lastSoldPrice,latitude,listPriceLow,livingArea,livingAreaUnits,livingAreaUnitsShort,livingAreaValue,longitude,lotAreaUnits,lotAreaValue,lotSize,nearbyZipcodes,neighborhoodRegion,parcelId,postingUrl,price,priceChange,priceChangeDate,priceChangeDateString,priceHistory,propertyTaxRate,propertyTypeDimension,providerListingID,regionString,rentZestimate,restimateHighPercent,restimateLowPercent,solarPotential,state,stateId,streetAddress,taxAssessedValue,taxAssessedYear,taxHistory,timeOnZillow,yearBuilt,zestimate,zestimateDeepDiveData,zestimateHighPercent,zestimateLowPercent,zipPlusFour,zipcode,zpid
0,5440 Bryant St,3.0,3.0,Denver,119221.0,Adams County,NaN,353.0,NaN,NaN,This 1600 square foot single family home has 3...,"[{'insights': [{'modelId': 'v2-2', 'phrases': ...",OTHER,SINGLE_FAMILY,NaN,NaN,39.795532,NaN,1600.0,Square Feet,sqft,1600.0,-105.019380,Square Feet,10200.0,10200.0,"[{'body': None, 'name': '80221', 'regionUrl': ...",NaN,NaN,NaN,586900.0,NaN,NaN,NaN,"[{'attributeSource': {'infoString1': None, 'in...",0.62,Single Family,NaN,Denver CO 80221,2997.0,7.0,12.0,NaN,CO,10.0,5440 Bryant St,NaN,NaN,[],NaN,"{'aboveGradeFinishedArea': None, 'accessibilit...",586900.0,{'zmaModel': None},9.0,8.0,1628.0,80221,2.126387e+09
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,331 34th Pl NE,1.0,3.0,Washington,41568.0,District of Columbia,11001.0,3289.0,NaN,NaN,This 992 square foot single family home has 3 ...,NaN,OTHER,SINGLE_FAMILY,NaN,NaN,38.894380,NaN,992.0,Square Feet,sqft,992.0,-76.956330,Square Feet,1474.0,1474.0,"[{'body': None, 'name': '20001', 'regionUrl': ...",{'name': 'River Terrace'},50180126,NaN,355900.0,NaN,NaN,NaN,[],0.55,Single Family,NaN,Washington DC 20019,2479.0,29.0,20.0,NaN,DC,12.0,331 34th Pl NE,348290.0,2023.0,"[{'taxIncreaseRate': -0.0019236632, 'taxPaid':...",NaN,"{'aboveGradeFinishedArea': None, 'accessibilit...",355900.0,{'zmaModel': None},10.0,13.0,1407.0,20019,5.092960e+05
3,4221 Sherrod St,3.0,3.0,Pittsburgh,26529.0,Allegheny County,42003.0,2614.0,1.689293e+12,102.0,Located in the heart of one of Pittsburgh’s ho...,"[{'insights': [{'modelId': 'v2-2', 'phrases': ...",RECENTLY_SOLD,SINGLE_FAMILY,NaN,537500.0,40.468243,NaN,2538.0,Square Feet,sqft,2538.0,-79.956055,Square Feet,2613.6,2613.0,"[{'body': None, 'name': '15206', 'regionUrl': ...",{'name': 'Central Lawrenceville'},0049G00150000000,NaN,537500.0,NaN,NaN,NaN,[{'attributeSource': {'infoString1': '1607301'...,1.62,Single Family,NaN,Pittsburgh PA 15201,3225.0,21.0,29.0,NaN,PA,47.0,4221 Sherrod St,316200.0,2023.0,"[{'taxIncreaseRate': 0, 'taxPaid': 1410.49, 't...",102 days,"{'aboveGradeFinishedArea': None, 'accessibilit...",542100.0,{'zmaModel': None},5.0,5.0,1733.0,15201,1.151351e+07
4,2206 Dawson Ave NE,1.0,2.0,Grand Rapids,11671.0,Kent County,26081.0,201.0,1.477008e+12,2559.0,Welcome to 2206 Dawson Ave NE on Grand Rapids ...,NaN,OTHER,SINGLE_FAMILY,NaN,72000.0,43.001930,NaN,754.0,Square Feet,sqft,754.0,-85.638210,Square Feet,6481.0,6481.0,"[{'body': None, 'name': '49503', 'regionUrl': ...",{'name': 'Creston'},411408452002,NaN,185700.0,NaN,NaN,NaN,[{'attributeSource': {'infoString1': '16026666...,1.14,Single Family,NaN,Grand Rapids MI 49505,1650.0,7.0,6.0,NaN,MI,30.0,2206 Dawson Ave NE,113000.0,2021.0,"[{'taxIncreaseRate': 0, 'taxPaid': 1980.51, 't...",2559 days,"{'aboveGradeFinishedArea': None, 'accessibilit...",185700.0,{'zmaModel': None},8.0,8.0,4380.0,49505,2.379746e+07
5,1112 S Adams St,5.0,5.0,Denver,11093.0,Denver County,8031.0,989.0,1.549411e+12,1721.0,This 4925 square foot single family home has 5...,NaN,OTHER,SINGLE_FAMILY,NaN,1485000.0,39.696278,NaN,4925.0,Square Feet,sqft,4925.0,-104.948500,Square Feet,6250.0,6

In [ ]:
original_years = pd.read_csv('/content/drive/MyDrive/test.csv')['yearBuilt']
test_df['yearBuilt'] = original_years
test_df.columns

<ipython-input-40-52c59e60dbc7>:1: DtypeWarning: Columns (56) have mixed types. Specify dtype option on import or set low_memory=False.
  original_years = pd.read_csv('/content/drive/MyDrive/test.csv')['yearBuilt']
<ipython-input-40-52c59e60dbc7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['yearBuilt'] = original_years


Index(['abbreviatedAddress', 'bathrooms', 'bedrooms', 'city', 'cityId',
       'county', 'countyFIPS', 'countyId', 'description', 'homeInsights',
       'homeStatus', 'homeType', 'lastSoldPrice', 'latitude', 'livingArea',
       'livingAreaUnits', 'livingAreaUnitsShort', 'livingAreaValue',
       'longitude', 'lotAreaUnits', 'lotAreaValue', 'lotSize', 'parcelId',
       'price', 'priceHistory', 'propertyTaxRate', 'propertyTypeDimension',
       'regionString', 'rentZestimate', 'state', 'stateId', 'streetAddress',
       'taxAssessedValue', 'taxHistory', 'yearBuilt', 'zestimate', 'zipcode',
       'zpid'],
      dtype='object')

In [ ]:
test_df.drop(columns=['description', 'homeInsights', 'homeStatus', 'livingAreaUnitsShort', 'livingAreaUnits', 'lotAreaUnits', 'priceHistory', 'propertyTypeDimension', 'taxHistory'])

,abbreviatedAddress,bathrooms,bedrooms,city,cityId,county,countyFIPS,countyId,homeType,lastSoldPrice,latitude,livingArea,livingAreaValue,longitude,lotAreaValue,lotSize,parcelId,price,propertyTaxRate,regionString,rentZestimate,state,stateId,streetAddress,taxAssessedValue,yearBuilt,zestimate,zipcode,zpid
0,5440 Bryant St,3.0,3.0,Denver,119221.0,Adams County,NaN,353.0,SINGLE_FAMILY,NaN,39.795532,1600.0,1600.0,-105.019380,10200.000000,10200.0,NaN,586900.0,0.62,Denver CO 80221,2997.0,CO,10.0,5440 Bryant St,NaN,"{'aboveGradeFinishedArea': None, 'accessibilit...",586900.0,80221,2.126387e+09
2,331 34th Pl NE,1.0,3.0,Washington,41568.0,District of Columbia,11001.0,3289.0,SINGLE_FAMILY,NaN,38.894380,992.0,992.0,-76.956330,1474.000000,1474.0,50180126,355900.0,0.55,Washington DC 20019,2479.0,DC,12.0,331 34th Pl NE,348290.0,"{'aboveGradeFinishedArea': None, 'accessibilit...",355900.0,20019,5.092960e+05
3,4221 Sherrod St,3.0,3.0,Pittsburgh,26529.0,Allegheny County,42003.0,2614.0,SINGLE_FAMILY,537500.0,40.468243,2538.0,2538.0,-79.956055,2613.600000,2613.0,0049G00150000000,537500.0,1.62,Pittsburgh PA 15201,3225.0,PA,47.0,4221 Sherrod St,316200.0,"{'aboveGradeFinishedArea': None, 'accessibilit...",542100.0,15201,1.151351e+07
4,2206 Dawson Ave NE,1.0,2.0,Grand Rapids,11671.0,Kent County,26081.0,201.0,SINGLE_FAMILY,72000.0,43.001930,754.0,754.0,-85.638210,6481.000000,6481.0,411408452002,185700.0,1.14,Grand Rapids MI 49505,1650.0,MI,30.0,2206 Dawson Ave NE,113000.0,"{'aboveGradeFinishedArea': None, 'accessibilit...",185700.0,49505,2.379746e+07
5,1112 S Adams St,5.0,5.0,Denver,11093.0,Denver County,8031.0,989.0,SINGLE_FAMILY,1485000.0,39.696278,4925.0,4925.0,-104.948500,6250.000000,6250.0,0524107002000,2185600.0,0.44,Denver CO 80210,8794.0,CO,10.0,1112 S Adams St,1560600.0,"{'aboveGradeFinishedArea': None, 'accessibilit...",2185600.0,80210,1.337194e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135608,4825 Quitman St,1.0,2.0,Denver,11093.0,Denver County,8031.0,989.0,SINGLE_FAMILY,NaN,39.784374,922.0,922.0,-105.040726,6250.000000,6250.0,0218427017000,518500.0,0.44,Denver CO 80212,2527.0,CO,10.0,4825 Quitman St,432500.0,"{'aboveGradeFinishedArea': None, 'accessibilit...",518500.0,80212.0,1.329373e+07
135609,1845 N Capitol St NE,0.0,NaN,Washington,41568.0,District of Columbia,11001.0,3289.0,MULTI_FAMILY,462000.0,38.915420,1894.0,1894.0,-77.008680,1310.000000,1310.0,35100022,968100.0,0.55,Washington DC 20002,1815.0,DC,12.0,1845 N Capitol St NE,1133030.0,"{'aboveGradeFinishedArea': None, 'accessibilit...",968100.0,20002.0,4.903460e+05
135610,115 S 13th St,2.0,3.0,Pittsburgh,26529.0,Allegheny County,42003.0,2614.0,SINGLE_FAMILY,92000.0,40.426700,2000.0,2000.0,-79.985030,1141.000000,1141.0,0003M00256000000,257800.0,1.62,Pittsburgh PA 15203,1998.0,PA,47.0,115 S 13th St,155700.0,"{'aboveGradeFinishedArea': None, 'accessibilit...",257800.0,15203.0,1.147467e+07
135611,2259 Chesapeake Dr NE,1.0,2.0,Grand Rapids,11671.0,Kent County,26081.0,201.0,SINGLE_FAMILY,140000.0,42.980347,998.0,998.0,-85.613820,0.253994,11064.0,411421228023,275900.0,1.14,Grand Rapids MI 49505,1688.0,MI,30.0,2259 Chesapeake Dr NE,177000.0,"{'aboveGradeFinishedArea': None, 'accessibilit...",275900.0,49505.0,2.380766e+07


In [ ]:
import re

def find_year(data_str):
  year_built = re.search(r"'factLabel': 'Year Built', 'factValue': '(\d{4})'", data_str)
  if year_built:
    return int(year_built.group(1))
  else:
    return None

In [ ]:
old_years = test_df['yearBuilt']
for year in old_years:
  year = find_year(str(year))

test_df['yearBuilt'] = old_years

<ipython-input-45-92c3b2ef6006>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['yearBuilt'] = old_years


In [ ]:
print(old_years)

0                                                      2007
2                                                      1939
3                                                      1906
4                                                      1930
5                                                      2017
                                ...                        
135608    {'aboveGradeFinishedArea': None, 'accessibilit...
135609    {'aboveGradeFinishedArea': None, 'accessibilit...
135610    {'aboveGradeFinishedArea': None, 'accessibilit...
135611    {'aboveGradeFinishedArea': None, 'accessibilit...
135612    {'aboveGradeFinishedArea': None, 'accessibilit...
Name: yearBuilt, Length: 120348, dtype: object


In [ ]:
test_df.head(10)

,abbreviatedAddress,bathrooms,bedrooms,city,cityId,county,countyFIPS,countyId,description,homeInsights,homeStatus,homeType,lastSoldPrice,latitude,livingArea,livingAreaUnits,livingAreaUnitsShort,livingAreaValue,longitude,lotAreaUnits,lotAreaValue,lotSize,parcelId,price,priceHistory,propertyTaxRate,propertyTypeDimension,regionString,rentZestimate,state,stateId,streetAddress,taxAssessedValue,taxHistory,yearBuilt,zestimate,zipcode,zpid
0,5440 Bryant St,3.0,3.0,Denver,119221.0,Adams County,NaN,353.0,This 1600 square foot single family home has 3...,"[{'insights': [{'modelId': 'v2-2', 'phrases': ...",OTHER,SINGLE_FAMILY,NaN,39.795532,1600.0,Square Feet,sqft,1600.0,-105.019380,Square Feet,10200.000000,10200.0,NaN,586900.0,"[{'attributeSource': {'infoString1': None, 'in...",0.62,Single Family,Denver CO 80221,2997.0,CO,10.0,5440 Bryant St,NaN,[],2007,586900.0,80221,2.126387e+09
2,331 34th Pl NE,1.0,3.0,Washington,41568.0,District of Columbia,11001.0,3289.0,This 992 square foot single family home has 3 ...,NaN,OTHER,SINGLE_FAMILY,NaN,38.894380,992.0,Square Feet,sqft,992.0,-76.956330,Square Feet,1474.000000,1474.0,50180126,355900.0,[],0.55,Single Family,Washington DC 20019,2479.0,DC,12.0,331 34th Pl NE,348290.0,"[{'taxIncreaseRate': -0.0019236632, 'taxPaid':...",1939,355900.0,20019,5.092960e+05
3,4221 Sherrod St,3.0,3.0,Pittsburgh,26529.0,Allegheny County,42003.0,2614.0,Located in the heart of one of Pittsburgh’s ho...,"[{'insights': [{'modelId': 'v2-2', 'phrases': ...",RECENTLY_SOLD,SINGLE_FAMILY,537500.0,40.468243,2538.0,Square Feet,sqft,2538.0,-79.956055,Square Feet,2613.600000,2613.0,0049G00150000000,537500.0,[{'attributeSource': {'infoString1': '1607301'...,1.62,Single Family,Pittsburgh PA 15201,3225.0,PA,47.0,4221 Sherrod St,316200.0,"[{'taxIncreaseRate': 0, 'taxPaid': 1410.49, 't...",1906,542100.0,15201,1.151351e+07
4,2206 Dawson Ave NE,1.0,2.0,Grand Rapids,11671.0,Kent County,26081.0,201.0,Welcome to 2206 Dawson Ave NE on Grand Rapids ...,NaN,OTHER,SINGLE_FAMILY,72000.0,43.001930,754.0,Square Feet,sqft,754.0,-85.638210,Square Feet,6481.000000,6481.0,411408452002,185700.0,[{'attributeSource': {'infoString1': '16026666...,1.14,Single Family,Grand Rapids MI 49505,1650.0,MI,30.0,2206 Dawson Ave NE,113000.0,"[{'taxIncreaseRate': 0, 'taxPaid': 1980.51, 't...",1930,185700.0,49505,2.379746e+07
5,1112 S Adams St,5.0,5.0,Denver,11093.0,Denver County,8031.0,989.0,This 4925 square foot single family home has 5...,NaN,OTHER,SINGLE_FAMILY,1485000.0,39.696278,4925.0,Square Feet,sqft,4925.0,-104.948500,Square Feet,6250.000000,6250.0,0524107002000,2185600.0,"[{'attributeSource': {'infoString1': None, 'in...",0.44,Single Family,Denver CO 80210,8794.0,CO,10.0,1112 S Adams St,1560600.0,"[{'taxIncreaseRate': 0.100162864, 'taxPaid': 8...",2017,2185600.0,80210,1.337194e+07
6,5433 McCandless Ave,1.0,2.0,Pittsburgh,26529.0,Allegheny County,42003.0,2614.0,This 1200 square foot single family home has 2...,NaN,OTHER,SINGLE_FAMILY,26300.0,40.479700,1200.0,Square Feet,sqft,1200.0,-79.945180,Square Feet,1400.000000,1400.0,0081B00154000000,154600.0,"[{'attributeSource': {'infoString1': None, 'in...",1.62,Single Family,Pittsburgh PA 15201,1414.0,PA,47.0,5433 McCandless Ave,64600.0,"[{'taxIncreaseRate': 0, 'taxPaid': 220.42, 'ti...",1900,154600.0,15201,1.161492e+07
7,1121 Dick Ave NW,1.0,3.0,Grand Rapids,11671.0,Kent County,26081.0,201.0,This 1021 square foot single family home has 3...,NaN,OTHER,SINGLE_FAMILY,80000.0,42.984780,1021.0,Square Feet,sqft,1021.0,-85.712240,Square Feet,10380.000000,10380.0,411322203019,240800.0,"[{'attributeSource': {'infoString1': None, 'in...",1.14,Single Family,Grand Rapids MI 49504,1798.0,MI,30.0,1121 Dick Ave NW,157000.0,"[{'taxIncreaseRate': 0, 'taxPaid': 1387.96, 't...",1965,240800.0,49504,2.377977e+07
8,1777 Sycamore St NW,4.0,4.0,Washington,41568.0,District of Columbia,11001.0,3289.0,This 2042 square foot single family home has 4...,NaN,OTHER,SINGLE_FAMILY,NaN,38.991386,2042.0,Square Feet,sqft,2042.0,-77.042010,Square Feet,87

In [ ]:
test_df = test_df.drop(columns=['description', 'homeInsights', 'homeStatus', 'livingAreaUnitsShort', 'livingAreaUnits', 'lotAreaUnits', 'priceHistory', 'propertyTypeDimension', 'taxHistory'])

In [ ]:
test_df.head(10)

,abbreviatedAddress,bathrooms,bedrooms,city,cityId,county,countyFIPS,countyId,homeType,lastSoldPrice,latitude,livingArea,livingAreaValue,longitude,lotAreaValue,lotSize,parcelId,price,propertyTaxRate,regionString,rentZestimate,state,stateId,streetAddress,taxAssessedValue,yearBuilt,zestimate,zipcode,zpid
0,5440 Bryant St,3.0,3.0,Denver,119221.0,Adams County,NaN,353.0,SINGLE_FAMILY,NaN,39.795532,1600.0,1600.0,-105.019380,10200.000000,10200.0,NaN,586900.0,0.62,Denver CO 80221,2997.0,CO,10.0,5440 Bryant St,NaN,2007,586900.0,80221,2.126387e+09
2,331 34th Pl NE,1.0,3.0,Washington,41568.0,District of Columbia,11001.0,3289.0,SINGLE_FAMILY,NaN,38.894380,992.0,992.0,-76.956330,1474.000000,1474.0,50180126,355900.0,0.55,Washington DC 20019,2479.0,DC,12.0,331 34th Pl NE,348290.0,1939,355900.0,20019,5.092960e+05
3,4221 Sherrod St,3.0,3.0,Pittsburgh,26529.0,Allegheny County,42003.0,2614.0,SINGLE_FAMILY,537500.0,40.468243,2538.0,2538.0,-79.956055,2613.600000,2613.0,0049G00150000000,537500.0,1.62,Pittsburgh PA 15201,3225.0,PA,47.0,4221 Sherrod St,316200.0,1906,542100.0,15201,1.151351e+07
4,2206 Dawson Ave NE,1.0,2.0,Grand Rapids,11671.0,Kent County,26081.0,201.0,SINGLE_FAMILY,72000.0,43.001930,754.0,754.0,-85.638210,6481.000000,6481.0,411408452002,185700.0,1.14,Grand Rapids MI 49505,1650.0,MI,30.0,2206 Dawson Ave NE,113000.0,1930,185700.0,49505,2.379746e+07
5,1112 S Adams St,5.0,5.0,Denver,11093.0,Denver County,8031.0,989.0,SINGLE_FAMILY,1485000.0,39.696278,4925.0,4925.0,-104.948500,6250.000000,6250.0,0524107002000,2185600.0,0.44,Denver CO 80210,8794.0,CO,10.0,1112 S Adams St,1560600.0,2017,2185600.0,80210,1.337194e+07
6,5433 McCandless Ave,1.0,2.0,Pittsburgh,26529.0,Allegheny County,42003.0,2614.0,SINGLE_FAMILY,26300.0,40.479700,1200.0,1200.0,-79.945180,1400.000000,1400.0,0081B00154000000,154600.0,1.62,Pittsburgh PA 15201,1414.0,PA,47.0,5433 McCandless Ave,64600.0,1900,154600.0,15201,1.161492e+07
7,1121 Dick Ave NW,1.0,3.0,Grand Rapids,11671.0,Kent County,26081.0,201.0,SINGLE_FAMILY,80000.0,42.984780,1021.0,1021.0,-85.712240,10380.000000,10380.0,411322203019,240800.0,1.14,Grand Rapids MI 49504,1798.0,MI,30.0,1121 Dick Ave NW,157000.0,1965,240800.0,49504,2.377977e+07
8,1777 Sycamore St NW,4.0,4.0,Washington,41568.0,District of Columbia,11001.0,3289.0,SINGLE_FAMILY,NaN,38.991386,2042.0,2042.0,-77.042010,8790.000000,8790.0,2759E0003,1199900.0,0.55,Washington DC 20012,5069.0,DC,12.0,1777 Sycamore St NW,1212630.0,1965,1199900.0,20012,4.702170e+05
9,69 Regina Ave,1.0,4.0,Hamilton,41279.0,Mercer County,34021.0,1201.0,SINGLE_FAMILY,300000.0,40.237410,1008.0,1008.0,-74.690510,7500.000000,7500.0,0301666000000069,300000.0,2.59,Hamilton NJ 08619,2764.0,NJ,40.0,69 Regina Ave,179800.0,1947,348700.0,08619,3.897022e+07
10,2324 Burning Tree Dr SE,2.5,4.0,Grand Rapids,11671.0,Kent County,26081.0,201.0,SINGLE_FAMILY,46000.0,42.921173,1371.0,1371.0,-85.605760,0.339991,14810.0,411810153008,385100.0,1.14,Grand Rapids MI 49546,2347.0,MI,30.0,2324 Burning Tree Dr SE,291200.0,1964,385100.0,49546,2.386298e+07


In [ ]:
test_df.shape[0]
test_df.columns

Index(['abbreviatedAddress', 'bathrooms', 'bedrooms', 'city', 'cityId',
       'county', 'countyFIPS', 'countyId', 'homeType', 'lastSoldPrice',
       'latitude', 'livingArea', 'livingAreaValue', 'longitude',
       'lotAreaValue', 'lotSize', 'parcelId', 'price', 'propertyTaxRate',
       'regionString', 'rentZestimate', 'state', 'stateId', 'streetAddress',
       'taxAssessedValue', 'yearBuilt', 'zestimate', 'zipcode', 'zpid'],
      dtype='object')

In [ ]:
test_df.to_csv('final_lsl_data.csv')